In [304]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer

In [305]:
categorical_schema = {
    'coffee_machine': {'9BARISTA': 0},
    'coffee_grinder': {'PIETRO': 0},
    'stove_type': {'CERAMIC': 0},
    'coffee_category': {
        'HUILA-COLOMBIA;PINK_BOURBON;ANAEROBIC_DOUBLE_FERMENTATION;LIGHT': 0,
        'JUKIA_PARK-UGANDA;SL-14;NATURAL_ANAEROBIC;MEDIUM': 1
    },
    'water_filtered': {False: 0}
}

inverted_categorical_schema = {
    category: {v: k for k, v in values.items()}
    for category, values in categorical_schema.items()
}

In [306]:
# Load from file
df = pd.read_csv('coffee_data.csv', sep=';')

optimal_bitterness = {
    "median": 5.6,
    "n_range": 0.6
} 
optimal_sourness = {
    "median": 5,
    "n_range": 0.5
}
get_bitterness_score = lambda n: 6 + (1 - (abs(optimal_bitterness["median"] - abs(n)) / optimal_bitterness["n_range"])) * 4 if abs(optimal_bitterness["median"] - abs(n)) < optimal_bitterness["n_range"] else 5 - np.clip((1 + abs(optimal_bitterness["median"] - abs(n)) ** 1.7), 0, 5)
get_sourness_score = lambda n: 6 + (1 - (abs(optimal_sourness["median"] - abs(n)) / optimal_sourness["n_range"])) * 4 if abs(optimal_sourness["median"] - abs(n)) < optimal_sourness["n_range"] else 5 - np.clip((1 + abs(optimal_sourness["median"] - abs(n)) ** 1.7), 0, 5)
get_end_score = lambda bitterness, sourness: (get_sourness_score(sourness) + get_bitterness_score(bitterness)) / 2


df['calculated_score'] = df.apply(lambda row: get_end_score(row['bitterness_score'], row['sourness_score']), axis=1)
df['coffee_category'] = df['coffee_origin'] + ';' + df['coffee_variety'] + ';' + \
                        df['coffee_processing_type'] + ';' + df['coffee_roast_level']

exclude_columns = [
    'coffee_machine', 
    'coffee_grinder', 
    'stove_type', 
    'heat_setting', 
    'coffee_origin',
    'coffee_variety', 
    'coffee_processing_type', 
    'coffee_roast_level', 
    'water_filtered',
    'water_weight', 
    'pre-infusion_time',
    'extraction_time', 
    'total_brew_time',
    'taste_score', 
    'aroma_intensity', 
    'bitterness_score', 
    'sourness_score', 
    'crema_quality'
]
df.drop(exclude_columns, axis=1, inplace=True)

# List of columns to encode
label_columns = [*{*categorical_schema}.difference(exclude_columns)]

# Encoding categorical columns
for col in label_columns:
    df[col] = [categorical_schema[col][value] for value in df[col]]

# Define Inputs and Outputs
inputs = [*{*['coffee_machine', 'coffee_grinder', 'stove_type', 'heat_setting', 'coffee_origin', 'coffee_variety', 'coffee_processing_type', 'coffee_roast_level', 'coffee_category', 'coffee_roasted_days_ago', 'coffee_dose', 'sprays_before_grinding', 'grind_size', 'water_filtered', 'water_weight']}.difference(exclude_columns)]
outputs = [*{*['pre-infusion_time', 'extraction_time', 'total_brew_time', 'taste_score', 'bitterness_score', 'sourness_score', 'aroma_intensity', 'crema_quality', 'calculated_score']}.difference(exclude_columns)]
columns_set = {*inputs, *outputs}

custom_scalers = {
    "default": MinMaxScaler(),
    "coffee_roasted_days_ago": MinMaxScaler().fit(pd.DataFrame({"coffee_roasted_days_ago": [0, 30]})),
    "coffee_dose": MinMaxScaler().fit(pd.DataFrame({"coffee_dose": [10, 25]})),
    "sprays_before_grinding": MinMaxScaler().fit(pd.DataFrame({"sprays_before_grinding": [0, 3]})),
    "grind_size": MinMaxScaler().fit(pd.DataFrame({"grind_size": [0.4, 1.5]})),
    "pre-infusion_time": MinMaxScaler().fit(pd.DataFrame({"pre-infusion_time": [0, 30]})),
    "extraction_time": MinMaxScaler().fit(pd.DataFrame({"extraction_time": [0, 30]})),
    "total_brew_time": MinMaxScaler().fit(pd.DataFrame({"total_brew_time": [120, 300]})),
    "taste_score": MinMaxScaler().fit(pd.DataFrame({"taste_score": [0, 10]})),
    "bitterness_score": MinMaxScaler().fit(pd.DataFrame({"bitterness_score": [0, 10]})),
    "sourness_score": MinMaxScaler().fit(pd.DataFrame({"sourness_score": [0, 10]})),
    "aroma_intensity": MinMaxScaler().fit(pd.DataFrame({"aroma_intensity": [0, 10]})),
    "crema_quality": MinMaxScaler().fit(pd.DataFrame({"crema_quality": [0, 10]})),
    "calculated_score": MinMaxScaler().fit(pd.DataFrame({"calculated_score": [0, 10]})),
}
for column in columns_set:
    if column in custom_scalers:
        df[column] = custom_scalers[column].transform(df[[column]])
        continue
    if column not in label_columns:
        df[column] = custom_scalers["default"].transform(df[[column]])

X = df[inputs]
y = df[outputs]

print(columns_set)

['coffee_category']
{'coffee_category', 'calculated_score', 'coffee_dose', 'sprays_before_grinding', 'coffee_roasted_days_ago', 'grind_size'}


In [307]:
# Model for hyperparameter tweaking
model = RandomForestRegressor(random_state=42)
param_grid = {
    'n_estimators': [i for i in range(50, 200, 10)],
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 3, 4, 5],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
}
loo = LeaveOneOut()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, n_jobs=5, scoring='neg_mean_squared_error')
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklea

In [ ]:
# Calculate LOOCV MSE as percentage error
y_true = y.values  # Actual values
y_pred = grid_search.predict(X)  # Predicted values

# Evaluate performance (using Mean Squared Error and R-squared)
mse = mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

# Calculate the mean of the actual values (for percentage calculation)
y_true_mean = np.mean(y_true)

# Calculate the percentage error
percentage_error = (mse / y_true_mean) * 100

print(f"Best Parameters: {best_params}")
print(f"Best Mean CV Error: {best_score}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared: {r2}")
print(f"Percentage Error: {percentage_error}%")
print()

Best Parameters: {'bootstrap': True, 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 90}
Best Mean CV Error: -0.03550914550252794
Mean Squared Error (MSE): 0.05055195637139968
Percentage Error: 17.012071786565194%



In [ ]:
# Train the model
# TODO: what amount of estimators do we need? 
best_model = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    max_features=best_params['max_features'],
    bootstrap=best_params['bootstrap'],
    random_state=42
)
errors = []
for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)
    errors.append(np.mean((y_test.values - y_pred) ** 2))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklea

In [ ]:
def objective(params):
    # Extract parameters from the input list
    coffee_category, coffee_dose, sprays_before_grinding, coffee_roasted_days_ago, grind_size = params

    # Create the input dataframe for prediction
    input_data = {
        'coffee_category': [coffee_category],  # Assuming this is already categorical or scaled
        'coffee_dose': custom_scalers["coffee_dose"].transform([[coffee_dose]])[0],
        'sprays_before_grinding': custom_scalers["sprays_before_grinding"].transform([[sprays_before_grinding]])[0],
        'coffee_roasted_days_ago': custom_scalers["coffee_roasted_days_ago"].transform([[coffee_roasted_days_ago]])[0],
        'grind_size': custom_scalers["grind_size"].transform([[grind_size]])[0],
    }

    input_df = pd.DataFrame(input_data)

    # Get model predictions (all outputs for the given input)
    prediction = best_model.predict(input_df)

    calculated_score = prediction[0]
    print(custom_scalers["calculated_score"].inverse_transform([[calculated_score]]))

    return -calculated_score

In [ ]:
# TODO: We can limit this to only contain the devices / coffee etc what we currently have, but still train the model on all the data :)
search_space = [
    Categorical([categorical_schema['coffee_category']['JUKIA_PARK-UGANDA;SL-14;NATURAL_ANAEROBIC;MEDIUM']], name='coffee_category'),
    Real(13, 22, name='coffee_dose'),
    Integer(0, 2, name='sprays_before_grinding'),
    Integer(9, 30, name='coffee_roasted_days_ago'),
    Real(0.6, 1.2, name='grind_size'),
]

In [ ]:
# Run Bayesian Optimization
# TODO: what amount of n_calls do we need?
result = gp_minimize(objective, search_space, n_calls=100, random_state=42)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feat

{'coffee_category': [1], 'coffee_dose': array([0.31006087]), 'sprays_before_grinding': array([0.66666667]), 'coffee_roasted_days_ago': array([0.73333333]), 'grind_size': array([0.42499968])}
[[3.77931523]]
{'coffee_category': [1], 'coffee_dose': array([0.47554934]), 'sprays_before_grinding': array([0.33333333]), 'coffee_roasted_days_ago': array([0.4]), 'grind_size': array([0.53684826])}
[[2.36818565]]
{'coffee_category': [1], 'coffee_dose': array([0.63319926]), 'sprays_before_grinding': array([0.66666667]), 'coffee_roasted_days_ago': array([0.3]), 'grind_size': array([0.72302449])}
[[2.27615321]]
{'coffee_category': [1], 'coffee_dose': array([0.5669919]), 'sprays_before_grinding': array([0.]), 'coffee_roasted_days_ago': array([0.3]), 'grind_size': array([0.46805891])}
[[2.27615321]]
{'coffee_category': [1], 'coffee_dose': array([0.2279994]), 'sprays_before_grinding': array([0.66666667]), 'coffee_roasted_days_ago': array([0.46666667]), 'grind_size': array([0.23123987])}
[[4.32102709]]
{

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feat

{'coffee_category': [np.int64(1)], 'coffee_dose': array([0.2]), 'sprays_before_grinding': array([0.66666667]), 'coffee_roasted_days_ago': array([1.]), 'grind_size': array([0.18181818])}
[[4.05038727]]
{'coffee_category': [np.int64(1)], 'coffee_dose': array([0.24226185]), 'sprays_before_grinding': array([0.66666667]), 'coffee_roasted_days_ago': array([0.3]), 'grind_size': array([0.72727273])}
[[3.9639551]]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feat

...

{'coffee_category': [np.int64(1)], 'coffee_dose': array([0.35084824]), 'sprays_before_grinding': array([0.66666667]), 'coffee_roasted_days_ago': array([0.43333333]), 'grind_size': array([0.18181818])}
[[4.32102709]]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


{'coffee_category': [np.int64(1)], 'coffee_dose': array([0.23824424]), 'sprays_before_grinding': array([0.66666667]), 'coffee_roasted_days_ago': array([0.4]), 'grind_size': array([0.18181818])}
[[4.32705957]]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


{'coffee_category': [np.int64(1)], 'coffee_dose': array([0.28815289]), 'sprays_before_grinding': array([0.66666667]), 'coffee_roasted_days_ago': array([0.8]), 'grind_size': array([0.72727273])}
[[3.77931523]]


In [ ]:
print("Best coffee brew settings found:")
for param_name, param_value in zip(search_space, result.x):
    if param_name.name in inverted_categorical_schema:
        param_value = inverted_categorical_schema[param_name.name][param_value]
    print(f"{param_name.name}: {param_value}")

print("Estimated final score: ", custom_scalers["calculated_score"].inverse_transform([[abs(result.fun)]])[0][0])

Best coffee brew settings found:
coffee_category: JUKIA_PARK-UGANDA;SL-14;NATURAL_ANAEROBIC;MEDIUM
coffee_dose: 13.366133416716231
sprays_before_grinding: 0
coffee_roasted_days_ago: 12
grind_size: 0.6
Estimated final score:  4.327059570882772
